In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score, validation_curve

C:\Users\siddh\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
X_train = []
X_test = []
Y_train = []
Y_test = []
X_valid = []
Y_valid = []
male = []
female = []

In [3]:
with open(r'data/training_set.txt') as training:
    for line in training:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_train.append(line_text)
            male.append(line_text)
            Y_train.append(0)
        elif(chr_gender.lower() == "f"):
            X_train.append(line_text)
            female.append(line_text)
            Y_train.append(1)


with open(r'data/test_set.txt') as test:
    for line in test:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_test.append(line_text)
            male.append(line_text)
            Y_test.append(0)
        elif(chr_gender.lower() == "f"):
            X_test.append(line_text)
            female.append(line_text)
            Y_test.append(1)




with open(r'data/validation_set.txt') as valid:
    for line in valid:
        line = line.strip()
        lnum, chr_id, movie_id, chr_name, chr_gender, line_text, credit_list = line.split("+++$+++")
        chr_gender = chr_gender.strip()
#         ss = sid.polarity_scores(line_text)
#         sentiment = ss['compound']
        if(chr_gender.lower() == "m"):
            X_valid.append(line_text)
            male.append(line_text)
            Y_valid.append(0)
        elif(chr_gender.lower() == "f"):
            X_valid.append(line_text)
            female.append(line_text)
            Y_valid.append(1)


In [4]:
X = X_train + X_test + X_valid
Y = Y_train + Y_test + Y_valid

In [5]:
indices = []
for i in range(len(X)):
  if(len(X[i]) > 0):
    answer = True 
  else:
    indices.append(i)

for index in sorted(indices, reverse=True):
    del X[index]
    del Y[index]

In [6]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X, Y = oversample.fit_resample(np.array(X).reshape(-1,1), Y)

In [7]:
a = []
for phrase in X:
    a.append(phrase[0].strip())

In [8]:
X = a
print(len(X))
print(type(X[0]))

341536
<class 'str'>


In [9]:
df = pd.DataFrame()
df['text'] = X
df['target'] = Y

In [10]:
test_df = pd.read_csv('data/tweets.csv', encoding="charmap")

In [11]:
test_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [12]:
test_data = test_df['text']
test_target = test_df['gender']

In [16]:
len(test_target) 

20050

In [17]:
len(test_data)

20050

In [14]:
clean_test_data = []
clean_test_target = []

In [18]:
for i in range(len(test_target)):
    element = test_target[i]
    text = test_data[i]
    if element == "male":
        clean_test_data.append(text)
        clean_test_target.append(0)
    elif element == "female":
        clean_test_data.append(text)
        clean_test_target.append(1)

In [19]:
len(clean_test_target)

12894

In [20]:
len(clean_test_data)

12894

In [22]:
train_data = df["text"]
train_target = df["target"]

In [23]:
test_data = clean_test_data
test_target = clean_test_target

In [24]:
bow_pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer()),
        ("classifier", MultinomialNB()),
    ]
)

bow_pipeline.fit(train_data, train_target)
y_pred = bow_pipeline.predict(test_data)
cr = classification_report(test_target, y_pred)

In [25]:
print(cr)


              precision    recall  f1-score   support

           0       0.52      0.72      0.60      6194
           1       0.60      0.39      0.47      6700

    accuracy                           0.55     12894
   macro avg       0.56      0.55      0.54     12894
weighted avg       0.56      0.55      0.53     12894



In [26]:
from sklearn.feature_extraction import DictVectorizer
class SegmentFeaturizer:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.future_words = ["tomorrow", "future", "futures"]
    
    @staticmethod
    def count_propernouns(doc):
        segment = doc.text.lower().split()
        count = 0 
        num = 0
        for token in doc:
            if token.tag_ in ['NNP', 'NNPS']:
                count+=1
                num += 1
            else:
                num += 1
        if(count == 0):
            average = 0
        else:
            average = count/num
        return average

    @staticmethod
    def get_n_words_before_main_verb(doc):
        numbers = [0]
        for sent in doc.sents:
            main = [t for t in sent if t.dep_ == "ROOT"][0]
            if main.pos_ == "VERB":
                dist_to_init = main.i - sent[0].i
                numbers.append(dist_to_init)
        return np.mean(numbers)

    @staticmethod
    def get_n_complex_clauses(doc):
        embedded_elements_count = []
        for sent in doc.sents:
            n_embedded = len(
                [t for t in sent if t.dep_ in {"ccomp", "xcomp", "advcl", "dative"}]
            )
            embedded_elements_count.append(n_embedded)
        if len(embedded_elements_count) == 0:
            return 0
        else:
            return np.mean(embedded_elements_count)
    
    @staticmethod
    def get_mean_sentiment(doc):
        return doc.sentiment
    @staticmethod
    def get_pronouns(doc):
        count = 0
        num = 0
        for token in doc:
            if token.pos_ == "PRON":
                count += 1
                num += 1
            else:
                num += 1
        if count == 0 or num == 0:
            average = 0
        else:
            average = count/num
        return average
    @staticmethod
    def get_female_pronouns(doc):
        count = 0
        num = 0
        for token in doc:
            if token.text.lower() in ['her', 'she', 'wife', 'girlfriend']:
                count += 1
                num += 1
            else:
                num+=1
        if count == 0 or num == 0:
            average = 0
        else:
            average = count/num
        return average
    @staticmethod
    def get_male_pronouns(doc):
        count = 0
        num = 0
        
        for token in doc:
            if token.text.lower() in ['he', 'him', 'husband', 'honey', 'his', 'boyfriend']:
                count += 1
                num += 1
            else:
                num += 1
        if count == 0 or num == 0:
            average = 0
        else:
            average = count/num
        return average
    def get_swear_words(doc):
        count = 0
        num = 0
        for token in doc:
            if token.lemma_.lower() in ["fuck", "shit", "bitch", "hell", "asshole", "ass"]:
                count += 1
                num += 1
            else:
                num += 1
        if count == 0 or num == 0:
            average = 0
        else:
            average = count/num
        return average
    # putting it all together!
    def featurize(self, segments):
        feature_dicts = []
        docs = self.nlp.pipe(segments)
        for doc in docs:
            feature_dict = {
                
                "n_propernouns": self.count_propernouns(doc),
                "n_words_before_main_verb": self.get_n_words_before_main_verb(doc),
                "n_complex_clauses": self.get_n_complex_clauses(doc),
                "mean_sentiment": self.get_mean_sentiment(doc),
                "n_pronouns": self.get_pronouns(doc),
                "n_male_pronouns": self.get_male_pronouns(doc),
                "n_female_pronouns": self.get_female_pronouns(doc)
                
            }
            feature_dicts.append(feature_dict)
        return feature_dicts
    

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin

segment_featurizer = SegmentFeaturizer()
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, data):
        return segment_featurizer.featurize(data)

In [29]:
manual_pipeline = Pipeline(
    steps=[
        ("stats", CustomLinguisticFeatureTransformer()),
        ("dict_vect", DictVectorizer()),
        ("classifier", LinearSVC()),
    ]
)

In [30]:
manual_pipeline.fit(train_data, train_target)
y_pred = manual_pipeline.predict(test_data)
crmanual = classification_report(test_target, y_pred)

C:\Users\siddh\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [34]:
print(crmanual)

              precision    recall  f1-score   support

           0       0.48      0.87      0.62      6194
           1       0.54      0.15      0.23      6700

    accuracy                           0.49     12894
   macro avg       0.51      0.51      0.43     12894
weighted avg       0.52      0.49      0.42     12894



In [31]:
bow_pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 3), stop_words='english')),
    ]
)

In [32]:
manual_pipeline = Pipeline(
    steps=[
        ("stats", CustomLinguisticFeatureTransformer()),
        ("dict_vect", DictVectorizer()),
    ]
)

In [33]:
embedding_pipeline = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
    ]
)

NameError: name 'SpacyVectorTransformer' is not defined

In [35]:
combined_features = FeatureUnion(
    transformer_list=[
        ("bow", bow_pipeline),
        ("manual", manual_pipeline),
    ]
)
final_pipeline = Pipeline(
    steps=[
        ("combined_features", combined_features),
        ("classifier",LinearSVC()),
    ]
)

In [36]:
final_pipeline.fit(train_data, train_target)
y_pred = final_pipeline.predict(test_data)
finalcrmultinomial = classification_report(test_target, y_pred)

C:\Users\siddh\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [37]:
print(finalcrmultinomial)

              precision    recall  f1-score   support

           0       0.50      0.72      0.59      6194
           1       0.55      0.33      0.41      6700

    accuracy                           0.51     12894
   macro avg       0.52      0.52      0.50     12894
weighted avg       0.53      0.51      0.50     12894



In [38]:
len(X_test)

25577